# Lab 01 Python for DS and AI - Supervised Learning - Regression

## Name Thantham Khamyai
## Student

# Import neccessary packages

In [1]:
import numpy as np
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

## Utility functions

In [2]:
# for avoiding repeatitive step of intercepts insertion, make function to do that
def add_intercept(X):
    ones_intercept = np.ones((X.shape[0], 1))
    return np.concatenate((ones_intercept, X), axis=1)

# Model class declaration

## Tasks completed

### 1. add option 'early_stopping' as boolean set in a model argument
### 2. 'stochastic' method option and investigating random select sample with non repeartitive until all
### 3. add option 'mini-batch' method with 'mini_batch_size' parameter can be defined 

In [3]:
class LinearRegression:
    
    def __init__(self, method='batch', max_iterations=10000, early_stopping=False,
                       alpha=.0001, tol=.00001, mini_batch_size=100, previous_loss=10000):
        self.method = method
        self.max_iterations = max_iterations
        self.early_stopping = early_stopping
        self.alpha = alpha
        self.tol = tol
        self.mini_batch_size = mini_batch_size
        self.previous_loss = previous_loss

        
    def fit(self, X, y):
        # 1. initalize theta
        self.theta = self.init_theta(X)
        
        # init blank used idx list for check repeatitive idx of stochastiac method
        idx_used = []
        
        # 2. loop along predefined n iterations
        for i in range(self.max_iterations):

            # 2.1 condition to choose method
            if self.method=='batch':
                # pass all samples
                x_to_train = X # dump all x
                y_to_train = y # dump sll y

            elif self.method=='stochastic':
                # randomly select 1 sample
                select_idx = np.random.randint(X.shape[0])# random idx
                while select_idx in idx_used:
                    select_idx = np.random.randint(X.shape[0])# random idx
                    
                x_to_train = np.array([X[select_idx, :]]) # extract one X by idx 
                y_to_train = np.array([y[select_idx]]) # extract one y by idx
                
                idx_used.append(select_idx)
                
                if len(idx_used) == X.shape[0]:
                    idx_used = []

            elif self.method=='mini-batch':
                # randomly select portion of samples following predefined mini batch size
                select_start_idx = np.random.randint(X.shape[0] - self.mini_batch_size) # random starting idx
                x_to_train = X[select_start_idx:select_start_idx + self.mini_batch_size, :] # extract portion of X
                y_to_train = y[select_start_idx:select_start_idx + self.mini_batch_size] # extract portion of y

            else:
                print('''wrong method defined 'batch','stochastic','mini-batch' only''')
                break

            # 2.2 predict y hat by dot x_to_train with theta
            yhat = self.predict(x_to_train)

            # 2.3 calculate error by minus yhst with y_to_train
            error = yhat - y_to_train

            # 2.4 calculate current mse to detect early stopping
            current_loss = self.mse(yhat, y_to_train)

            # 2.5 if early stopping set as True & difference of current and previous loss is less than threshold
            if self.early_stopping & (np.abs(self.previous_loss - current_loss) < self.tol):

                # print early stopped epoch and exit loop
                print(f'early_stopped at epoch: {i+1}')
                break

            # 2.6 if not early stop or set False, update previous loss
            self.previous_loss = current_loss

            # 2.7 calculate gradient of trainingdata
            grad = self.gradient(x_to_train, error)

            # 2.8 update theta
            self.theta = self.theta - self.alpha * grad
        
    # function to predict yhat   
    def predict(self, X):
        return X @ self.theta

    # function to calculate loss
    def mse(self, yhat, y):
        return ((yhat - y)**2).sum() / yhat.shape[0]

    # function to calculate gradient
    def gradient(self, X, error):
        return X.T @ error

    # function to create initial theta
    def init_theta(self, X):
        return np.zeros((X.shape[1]))        


# Model Testing

## 1 Load Data Into X, y

In [4]:
boston = load_boston()

X = boston.data
y = boston.target

print(f'number of samples (m): {X.shape[0]}')
print(f'number of features (n): {X.shape[1]}')
print(f'shape of x: {X.shape}')
print(f'shape of y: {y.shape}')

number of samples (m): 506
number of features (n): 13
shape of x: (506, 13)
shape of y: (506,)


## 2 Scaling Data

In [5]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

## 3 Add Intercept Term 

In [6]:
X = add_intercept(X)

print(f'shape of X after insert intercepts: {X.shape}')

shape of X after insert intercepts: (506, 14)


## 4 Train Test Splitting

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)

print(f'shape of X_train: {X_train.shape}')
print(f'shape of X_test: {X_test.shape}')
print(f'shape of y_train: {y_train.shape}')
print(f'shape of y_test: {y_test.shape}')

shape of X_train: (354, 14)
shape of X_test: (152, 14)
shape of y_train: (354,)
shape of y_test: (152,)


## 5 Model Instance Create and Fitting

In [8]:
# selective methods : 'batch', 'mini-batch', 'stochastic'

model = LinearRegression(method='batch', max_iterations=300000, early_stopping=True, alpha=0.0001, tol=0.00001, mini_batch_size=50)
model.fit(X_train, y_train)

early_stopped at epoch: 1224


## 6 Predict y and Show MSE

In [9]:
y_pred = model.predict(X_test)
mse = model.mse(y_pred, y_test)

print(f'MSE after fitting: {mse}')

MSE after fitting: 21.08105356440145
